In [1]:
import pandas as pd
import json
import configparser
from pathlib import Path
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')
data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses a serem previstos

# Prever Cesta

In [2]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes_cesta' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

Epoch 1/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0201
Epoch 2/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0125
Epoch 3/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0076
Epoch 4/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0044
Epoch 5/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 6/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0034
Epoch 7/150
197/197 [==============================] - 0s 3ms/step - loss: 0.0026
Epoch 8/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 9/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0018
Epoch 10/150
197/197 [==============================] - 0s 3ms/step - loss: 0.0016
Epoch 11/150
197/197 [==============================] - 1s 3ms/step - loss: 0.0014
Epoch 12/150
197/197 [==============================] - 0s 2ms/step - loss: 0.0013
Epoch 13/150


# Prever Produtos

In [2]:
path_inicial = r'C:\\Users\\mathe\\Documents\\NotebooksMateriais\\previsao_cestas\\datasets_produtos\\'

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        resultados = [valor for valor in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(resultados)}
        filename = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
        pasta_atual = os.getcwd()
        full_url = Path(pasta_atual) / 'resultados' / 'previsoes_produtos' / f'{regiao}' / filename
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")

Epoch 1/150
197/197 [==============================] - 0s 1ms/step - loss: 1.5838e-05
Epoch 2/150
197/197 [==============================] - 0s 1ms/step - loss: 7.5648e-06
Epoch 3/150
197/197 [==============================] - 0s 1ms/step - loss: 7.3683e-06
Epoch 4/150
197/197 [==============================] - 0s 1ms/step - loss: 6.6879e-06
Epoch 5/150
197/197 [==============================] - 0s 998us/step - loss: 7.1591e-06
Epoch 6/150
197/197 [==============================] - 0s 1ms/step - loss: 4.7639e-06
Epoch 7/150
197/197 [==============================] - 0s 1ms/step - loss: 5.0320e-06
Epoch 8/150
197/197 [==============================] - 0s 1ms/step - loss: 4.8208e-06
Epoch 9/150
197/197 [==============================] - 0s 1ms/step - loss: 5.2751e-06
Epoch 10/150
197/197 [==============================] - 0s 996us/step - loss: 7.7001e-06
Epoch 11/150
197/197 [==============================] - 0s 995us/step - loss: 5.8791e-06
Epoch 12/150
197/197 [========================

197/197 [==============================] - 0s 1ms/step - loss: 3.3414e-06
Epoch 96/150
197/197 [==============================] - 0s 1ms/step - loss: 3.9104e-06
Epoch 97/150
197/197 [==============================] - 0s 1ms/step - loss: 3.0813e-06
Epoch 98/150
197/197 [==============================] - 0s 1ms/step - loss: 3.3666e-06
Epoch 99/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4140e-06
Epoch 100/150
197/197 [==============================] - 0s 1ms/step - loss: 3.2365e-06
Epoch 101/150
197/197 [==============================] - 0s 1ms/step - loss: 3.3470e-06
Epoch 102/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4105e-06
Epoch 103/150
197/197 [==============================] - 0s 1ms/step - loss: 3.2277e-06
Epoch 104/150
197/197 [==============================] - 0s 1ms/step - loss: 3.2513e-06
Epoch 105/150
197/197 [==============================] - 0s 975us/step - loss: 3.3062e-06
Epoch 106/150
197/197 [=========================

197/197 [==============================] - 0s 1ms/step - loss: 3.7950e-06
Epoch 38/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5819e-06
Epoch 39/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5174e-06
Epoch 40/150
197/197 [==============================] - 0s 1ms/step - loss: 3.6120e-06
Epoch 41/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4796e-06
Epoch 42/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4670e-06
Epoch 43/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5634e-06
Epoch 44/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5335e-06
Epoch 45/150
197/197 [==============================] - 0s 1ms/step - loss: 3.6902e-06
Epoch 46/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4357e-06
Epoch 47/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4333e-06
Epoch 48/150
197/197 [==============================] - 

Epoch 131/150
197/197 [==============================] - 0s 1ms/step - loss: 4.9306e-06
Epoch 132/150
197/197 [==============================] - 0s 965us/step - loss: 4.4894e-06
Epoch 133/150
197/197 [==============================] - 0s 963us/step - loss: 5.1601e-06
Epoch 134/150
197/197 [==============================] - 0s 966us/step - loss: 4.6558e-06
Epoch 135/150
197/197 [==============================] - 0s 964us/step - loss: 7.6118e-06
Epoch 136/150
197/197 [==============================] - 0s 1ms/step - loss: 4.7785e-06
Epoch 137/150
197/197 [==============================] - 0s 1ms/step - loss: 5.2103e-06
Epoch 138/150
197/197 [==============================] - 0s 1ms/step - loss: 4.0130e-06A: 0s - loss: 3.6774e-0
Epoch 139/150
197/197 [==============================] - 0s 1ms/step - loss: 4.5471e-06
Epoch 140/150
197/197 [==============================] - 0s 1ms/step - loss: 4.2105e-06
Epoch 141/150
197/197 [==============================] - 0s 1ms/step - loss: 4.2248e-06
E

Epoch 73/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5231e-05
Epoch 74/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5187e-05
Epoch 75/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5146e-05
Epoch 76/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5109e-05
Epoch 77/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5074e-05
Epoch 78/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5042e-05
Epoch 79/150
197/197 [==============================] - 0s 1ms/step - loss: 3.5013e-05
Epoch 80/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4987e-05
Epoch 81/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4963e-05
Epoch 82/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4941e-05
Epoch 83/150
197/197 [==============================] - 0s 1ms/step - loss: 3.4920e-05
Epoch 84/150
197/197 [=====================

...

197/197 [==============================] - 0s 1ms/step - loss: 1.7999e-06
Epoch 44/150
197/197 [==============================] - 0s 1ms/step - loss: 1.8048e-06
Epoch 45/150
197/197 [==============================] - 0s 1ms/step - loss: 1.8058e-06
Epoch 46/150
197/197 [==============================] - 0s 1ms/step - loss: 2.3434e-06
Epoch 47/150
197/197 [==============================] - 0s 1ms/step - loss: 1.8461e-06
Epoch 48/150
197/197 [==============================] - 0s 1ms/step - loss: 1.5511e-06
Epoch 49/150
197/197 [==============================] - 0s 1ms/step - loss: 1.5021e-06A: 0s - loss: 2.7519e
Epoch 50/150
197/197 [==============================] - 0s 1ms/step - loss: 1.5379e-06
Epoch 51/150
197/197 [==============================] - 0s 2ms/step - loss: 1.4865e-06
Epoch 52/150
197/197 [==============================] - 0s 1ms/step - loss: 1.5994e-06
Epoch 53/150
197/197 [==============================] - 0s 1ms/step - loss: 1.4655e-06
Epoch 54/150
197/197 [=============

197/197 [==============================] - 0s 1ms/step - loss: 5.1326e-07
Epoch 136/150
197/197 [==============================] - 0s 1ms/step - loss: 5.1612e-07
Epoch 137/150
197/197 [==============================] - 0s 1ms/step - loss: 5.0843e-07
Epoch 138/150
197/197 [==============================] - 0s 1ms/step - loss: 5.0811e-07
Epoch 139/150
197/197 [==============================] - 0s 1ms/step - loss: 5.0535e-07
Epoch 140/150
197/197 [==============================] - 0s 1ms/step - loss: 5.0880e-07
Epoch 141/150
197/197 [==============================] - 0s 1ms/step - loss: 5.1412e-07
Epoch 142/150
197/197 [==============================] - 0s 1ms/step - loss: 5.2569e-07
Epoch 143/150
197/197 [==============================] - 0s 1ms/step - loss: 5.4556e-07
Epoch 144/150
197/197 [==============================] - 0s 1ms/step - loss: 5.8259e-07
Epoch 145/150
197/197 [==============================] - 0s 1ms/step - loss: 6.6591e-07
Epoch 146/150
197/197 [=======================

197/197 [==============================] - 0s 1ms/step - loss: 9.2987e-06
Epoch 70/150
197/197 [==============================] - 0s 1ms/step - loss: 8.3632e-06
Epoch 71/150
197/197 [==============================] - 0s 1ms/step - loss: 8.4159e-06
Epoch 72/150
197/197 [==============================] - 0s 1ms/step - loss: 8.2494e-06
Epoch 73/150
197/197 [==============================] - 0s 1ms/step - loss: 8.1791e-06
Epoch 74/150
197/197 [==============================] - 0s 1ms/step - loss: 8.5662e-06
Epoch 75/150
197/197 [==============================] - 0s 1ms/step - loss: 8.2352e-06
Epoch 76/150
197/197 [==============================] - 0s 1ms/step - loss: 9.0560e-06
Epoch 77/150
197/197 [==============================] - 0s 1ms/step - loss: 8.6660e-06A: 0s - loss: 1.0050e-
Epoch 78/150
197/197 [==============================] - 0s 1ms/step - loss: 8.9531e-06
Epoch 79/150
197/197 [==============================] - 0s 1ms/step - loss: 8.6890e-06
Epoch 80/150
197/197 [============

197/197 [==============================] - 0s 1ms/step - loss: 5.8981e-04
Epoch 4/150
197/197 [==============================] - 0s 1ms/step - loss: 3.9105e-04
Epoch 5/150
197/197 [==============================] - 0s 1ms/step - loss: 2.9197e-04
Epoch 6/150
197/197 [==============================] - 0s 1ms/step - loss: 2.4148e-04A: 0s - loss: 2.2601e-
Epoch 7/150
197/197 [==============================] - 0s 1ms/step - loss: 2.1697e-04
Epoch 8/150
197/197 [==============================] - 0s 1ms/step - loss: 2.0283e-04
Epoch 9/150
197/197 [==============================] - 0s 1ms/step - loss: 1.9763e-04
Epoch 10/150
197/197 [==============================] - 0s 1ms/step - loss: 1.8861e-04
Epoch 11/150
197/197 [==============================] - 0s 1ms/step - loss: 1.8653e-04
Epoch 12/150
197/197 [==============================] - 0s 1ms/step - loss: 1.8004e-04
Epoch 13/150
197/197 [==============================] - 0s 1ms/step - loss: 1.6858e-04
Epoch 14/150
197/197 [==================

197/197 [==============================] - 0s 1ms/step - loss: 1.1157e-04
Epoch 97/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1123e-04
Epoch 98/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1050e-04
Epoch 99/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1064e-04
Epoch 100/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1050e-04
Epoch 101/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1144e-04
Epoch 102/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1081e-04
Epoch 103/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1030e-04
Epoch 104/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1089e-04
Epoch 105/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1129e-04
Epoch 106/150
197/197 [==============================] - 0s 1ms/step - loss: 1.1046e-04
Epoch 107/150
197/197 [==========================